### SWIMPLACES 

In [251]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import unicodedata
import numpy as np
from tqdm import tqdm

In [252]:
#how to access another than the first page
response = requests.get('https://www.mapotic.com/api/v1/maps/2941/public-pois/?page=1&page_size=10&ordering=created&page=4')
print(response)
#response.json()

<Response [200]>


In [253]:
class Swim():
    '''
    This Class will be used to download all data and merge them into one DataFrame.
    '''
    def __init__(self):
        self.pages_json = self.downloadPages()
        self.get_data = self.getData()
        pass
    def downloadPages(self):
        """Download data from API and save to a json file."""
        
        resp=[]
        b= tqdm([1,2], desc="Downloading JSON data from API")  # MAZU 2 STRANKY PRO RYCHLOST a mensim page_size z 1000 na 50!!
        for i in b:
            response = requests.get('https://www.mapotic.com/api/v1/maps/2941/public-pois/?page=1&page_size=20&ordering=created&page='+ str(i) )
            resp.append(response.json())
#         with open('raw_data.json', 'w') as raw:   # uložit do classy, ale aby se to pokaždé nestahovalo... plus podmínku, kdy se má stáhnout
#             json.dump(resp, raw) 
        return resp
            
    def getData(self):
        """Extract available data from JSON"""
        preview = self.pages_json
#         with open('raw_data.json') as raw_data: 
#             preview = json.load(raw_data)
        latitude = []    
        longitude = []
        ids = [] 
        names = [] 
#         types =[]
        ratings = []
        count_ratings = []
        created = []
        for page in preview: 
            for result in page['results']:
                names.append(result['name'])
                ids.append(result['id'])
                latitude.append(result['point']['coordinates'][0])
                longitude.append(result['point']['coordinates'][1])
#                 types.append(result['category']['name']['en'][0])  #z nějakého důvodu nefunguje - NoneType error
                ratings.append(result['rating']['average'])
                count_ratings.append(result['rating']['count'])
                created.append(result['created'])

        df = pd.DataFrame({
            'ID' : ids, 
            'Name' : names, 
#             'type' : types,
            'Average rating' : ratings,
            'Number of ratings' : count_ratings,
            'Created' : created,
            'Longitude' : longitude,
            'Latitude' : latitude})
#         return df 
        return df.loc[:30]  #zde ukáže momentálně pár záznamů, jen pro kontrolu
    
    def getDescPlaces(self, idn, name):
        """Function to get the source (only the part we are interested in) from each specific website. ID and name (without diacritics) have to be entered."""
        
        name = unicodedata.normalize('NFKD', name).lower()
        new_name = ''
        for c in name:
            if not unicodedata.combining(c):
                new_name += c
                
        odkaz = 'https://www.swimplaces.com/'  + str(idn) + '-'+ new_name.replace(' ', '-') 
        r = requests.get(odkaz)
        r.encoding = 'UTF-8'
        soup = BeautifulSoup(r.text,'lxml').find('meta', {'name' : 'description'})['content'].split('|')
        return soup
    
    def getAtt(self):
        """Download attributes for all the places."""
        value = None
        def find_between(string, start, end):  #pomocná na hledání ve stringu
            return (string.split(start))[1].split(end)[0]
        
        nazev =[]
        att = []
        for i, n in self.get_data.iterrows(): #self.getData().iterrows(): 
            nazev.append(self.getDescPlaces(n['ID'], n['Name'])[0])
            try:
                att.append(self.getDescPlaces(n['ID'], n['Name'])[1])  # tady už to někdy hodilo chybu, ze to nenašlo...
            except:
                att.append(value)
        ids_att = self.get_data['ID'].values #self.getData()['ID'].values #přidat ID, pro pozdější join
        
        desc = []
        refresh = []
        diving = []
        entrances = []
        access = []
        nudists = []
        for ii in tqdm(att, desc = 'Searching through the description for attributes:'): # tady zkusim pridat taky tqdm
            if 'Description:' in ii:
                desc.append(find_between(ii, 'Description:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip()) # najde atribut a vrátí jeho obsah - snad nikde jinde není dvojtečka :D
            else:
                desc.append(value)
            if 'Refreshment' in ii:
                refresh.append(find_between(ii, 'Refreshment:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip())
            else:
                refresh.append(value) 
            if 'Diving' in ii:
                diving.append(find_between(ii, 'Diving:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip())
            else:
                diving.append(value)
            if 'Accessibility/parking' in ii:
                access.append(find_between(ii, 'Accessibility/parking:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip()) 
            else:
                access.append(value)                                                         
            if 'Entrance' in ii:
                entrances.append(find_between(ii, 'Entrance:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip()) 
            else:
                entrances.append(value)
            if 'Nudist beach' in ii:
                nudists.append(find_between(ii, 'Nudist beach:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip())
            else: 
                nudists.append(value)
        attributes = pd.DataFrame({
            'id_a' : ids_att,
            'nazev' :nazev,
            'Description' : desc,
            'Refreshment' : refresh,
            'Diving' : diving,
            'Entrance' : entrances,
            'Accessibility and parking' :access,
            'Nudist beach' : nudists
        })
        return attributes
    def mergeData(self):
        ''' Merge those 2 data sets, cleaning will follow.'''
        output = self.get_data.merge(self.getAtt(), left_on = 'ID', right_on = 'id_a')
#         output = output['ids', 'names'] # , 'avg_rating', 'count_ratings', 'created', 'logitude', 'latitude', 'description', 'refreshment', 'diving', 'entrance', 'accessibility_parking', 'nudist_beach']
        output.to_csv('raw_data.csv', sep = ',')
        return print('Data prepared in csv file.')

Nevim, proč se druhý tqdm neukazuje v prubehu - a jak dlouho trva celkove stazeni?
- doresit
- pridat poznamky

----------

In this Jupyter notebook, we interpret and visualize data collected in previous notebook.

### Import of packages

In this section, we import all packages that are necessary for *data interpretation* as well as for their *visualization*. First, we import pandas, geopandas, json, numpy and matpotlib.pyplot in order to process the data. Then, we import Nominatim from geopy.geocoders in order to determine the location of the user. To perform geodetic computations, we import Geod from pyproj. Finally, we import folium and its plugin to visualize data.

In [577]:
import pandas as pd
import geopandas as gpd
import json
import numpy as np
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim

from pyproj import Geod

#python -m pip install folium
import folium
from folium.plugins import MarkerCluster

### Data loading & descriptive statistics

After importing packages, it is necessary to download json data from API. Data are then merged and prepared in csv file.

In [717]:
swim = Swim()
# swim.downloadPages() # update soup - change  -> už není potreba
swim.mergeData() #- > vytvoreni csv souboru
swim_data = pd.read_csv('raw_data.csv')

Searching through the description for attributes:: 100%|██████████| 20/20 [00:00<00:00, 11210.22it/s]

Data prepared in csv file.


We provide desriptive statistics, including only numeric columns. Since column *ID* and column *id_a* (which are identical) are not useful for the purpose of data description, we remove both of them.

In [709]:
swim_desc = swim_data.drop('id_a', 1).drop('ID', 1).drop('Unnamed: 0', 1)
desc = swim_desc.describe(include = [np.number])

desc

,Average rating,Number of ratings,Longitude,Latitude
count,18.000000,20.000000,20.000000,20.000000
mean,4.168681,4.200000,49.114230,14.603510
std,0.834919,3.427443,2.100714,3.052217
min,2.000000,0.000000,41.760320,2.983775
25%,3.892857,1.750000,49.065163,14.315799
50%,4.387500,3.500000,50.001197,14.636598
75%,4.766667,5.500000,50.152053,15.829210
max,5.000000,12.000000,50.279876,19.445608


To prepare data for another part, we will select only columns with Name, Latitude, Longitude and Average rating of each location. We display first rows of our data to make sure that they are in desired format.

In [735]:
swim_loc = swim_data[['Name','Latitude','Longitude', 'Average rating', 'Entrance', 'Nudist beach']]
swim_loc.head()

,Name,Latitude,Longitude,Average rating,Entrance,Nudist beach
0,Costa Brava,2.983775,41.760320,NaN,No entrance fee,Not suitable for nudists
1,Nádrž Dolní Žleb,17.307842,49.751182,2.0,No entrance fee,Not suitable for nudists
2,Kralupy nad Vltavou,14.307153,50.256161,5.0,Entrance fee,Not suitable for nudists
3,Koupaliště Flošna,15.841977,50.205154,4.8,Entrance fee,Suitable for nudists
4,Městské lázně,15.828759,50.214217,4.5,Entrance fee,Not suitable for nudists


To find a set of coordinates corresponding to the address, we use Nominatim imported from geopy.geocoders. Nominatim uses OpenStreetMap data to find any locations on Earth by name and address (geocoding) and to detect its coordinates. It can also do the reverse. 

Any user of this application can insert his location (city is enough) into the brackets and the application will determine precise address as well as the coordinates of that place.

In [736]:
geolocator = Nominatim(user_agent='myapplication')

# Insert city of your location:
location = geolocator.geocode("Litomyšl")

print("Your location is ", location.address, ".")
print("Longitude = {}, Latitude = {}".format(location.longitude, location.latitude))

Your location is  Litomyšl, okres Svitavy, Pardubický kraj, Severovýchod, 570 01, Česká republika .
Longitude = 16.3101243, Latitude = 49.8725491


Ater we know location of user, the application can measure the distance between that location and all the places in the dataset. We decided to use World Geodetic System (WGS) ellipsoid to measure the distances as it is a highly accurate approach.

In [737]:
#We use World Geodetic System ellipsoid to measure the distance.
g = Geod(ellps='WGS84')

#We declare a function measuring distance between pairs of latitude-longitude points.
def Distance(name, lat1, lon1, lat2, lon2, rating):
    az12,az21,dist = g.inv(lon1, lat1, lon2, lat2)
    return dist

#As lat1 and lon1, we will use latitude and longitude of user's location.
name = swim_loc['Name']
lat1 = location.latitude
lon1 = location.longitude
lat2 = swim_loc['Latitude']
lon2 = swim_loc['Longitude']
rating = swim_loc['Average rating']

#We define new dataframe including values of columns from swim_loc.
df = pd.DataFrame({'Name':name,'Average rating':rating,'Latitude_1':lat1,'Longitude_1':lon1,'Latitude_2':lat2,'Longitude_2':lon2})

#We add a column with distance in metres using distance function.
df['Distance'] = Distance(df['Name'].tolist(),df['Latitude_1'].tolist(),df['Longitude_1'].tolist(),df['Latitude_2'].tolist(),df['Longitude_2'].tolist(),df['Average rating'].tolist())

#Then we convert column with distance to kilometers and display first rows of new dataframe.
df['Distance'] = (1/1000)*df['Distance']

df.head()

,Name,Average rating,Latitude_1,Longitude_1,Latitude_2,Longitude_2,Distance
0,Costa Brava,NaN,49.872549,16.310124,2.983775,41.760320,5725.521722
1,Nádrž Dolní Žleb,2.0,49.872549,16.310124,17.307842,49.751182,4688.608866
2,Kralupy nad Vltavou,5.0,49.872549,16.310124,14.307153,50.256161,5000.285866
3,Koupaliště Flošna,4.8,49.872549,16.310124,15.841977,50.205154,4852.033516
4,Městské lázně,4.5,49.872549,16.310124,15.828759,50.214217,4853.787915


We select 5 geographically nearest places to user's location and sort those places by the average rating.

In [738]:
#We define new dataframe with 5 geographicaly nearest places to user's location. 
df_map = df.nsmallest(5, columns=['Distance']).sort_values(['Average rating'], ascending=[True])

#We remove columns Latitude 1 and Longitude 1 as those coordinates are identical for all rows and we don't need them.
df_map.drop(['Latitude_1','Longitude_1'], axis=1, inplace=True)

#Then, we reset indexing of rows for easier manipulation with data in future steps.
df_map.reset_index(drop=True, inplace=True)

#We round values in column Average rating and Distance to one decimal point and display the result.
df_map['Average rating'] = df_map['Average rating'].round()
df_map['Distance'] = df_map['Distance'].round()

df_map

,Name,Average rating,Latitude_2,Longitude_2,Distance
0,Nádrž Dolní Žleb,2.0,17.307842,49.751182,4689.0
1,Plavecký bazén Ústí nad Orlicí,4.0,16.402715,49.975747,4786.0
2,Termální koupaliště Bešeňová,4.0,19.445608,49.100166,4452.0
3,Sluneční pláž,5.0,15.782788,50.033879,4848.0
4,Romantická pláž,5.0,14.768348,44.922321,4676.0


### Data visualization

In the final part, we will analyze and visualize collected and summarized data.

First, we will map all the points from the original dataset, using the Folium module. Since most of the places are located in the Czech Republic, we put latitude and longitude of Czech Republic as the initial coordinates of the map.

In [739]:
#We need to remove punctuation from 'Name' column since the map does not display those letters correctly.
swim_loc['Name'] = swim_loc['Name'].str.replace('á','a').str.replace('í','i').str.replace('é','e').str.replace('ě','e').str.replace('š','s').str.replace('Š','S').str.replace('č','c').str.replace('Č','C').str.replace('ř','r').str.replace('Ř','R').str.replace('ž','z').str.replace('Ž','Z').str.replace('ý','y').str.replace('ů','u').str.replace('ú','u').str.replace('Ú','U').str.replace('ň','n')

#We create map object using Folium. Based on our preferences, we set tiles to cartodbpositron and zoom_start to 8.
map0 = folium.Map(
    location=[49.8037633, 15.4749126],
    tiles='cartodbpositron',
    zoom_start=8)

#Then we create a list of latitude and longitude coordinate pairs.
map_locations0 = swim_loc[['Latitude', 'Longitude']]
locationlist0 = map_locations0.values.tolist()

#We set parameters of markers so that all points are well visible and recognizable. We add these markers to our map.
for point in range(0, len(locationlist0)):
    folium.Marker(locationlist0[point], 
                  popup=swim_loc['Name'][point], 
                  icon=folium.Icon(color='darkblue', icon='tint')
                 ).add_to(map0)    
    
#We create marker clusters that group points that overlap.
marker_cluster = folium.plugins.MarkerCluster().add_to(map0)    
    
map0

/Users/Andrea/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


...

In [740]:
#We need to remove punctuation from 'Name' column since the map does not display those letters correctly.
df_map['Name'] = df_map['Name'].str.replace('á','a').str.replace('í','i').str.replace('é','e').str.replace('ě','e').str.replace('š','s').str.replace('Š','S').str.replace('č','c').str.replace('Č','C').str.replace('ř','r').str.replace('Ř','R').str.replace('ž','z').str.replace('Ž','Z').str.replace('ý','y').str.replace('ů','u').str.replace('ú','u').str.replace('Ú','U').str.replace('ň','n')


#We create map object using Folium. Based on our preferences, we set tiles to cartodbpositron and zoom_start to 10.
map1 = folium.Map(
    location=[location.latitude, location.longitude],
    tiles='cartodbpositron',
    zoom_start=10)

#Then we create a list of latitude and longitude coordinate pairs.
map_locations = df_map[['Latitude_2', 'Longitude_2']]
locationlist = map_locations.values.tolist()


for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_map['Name'][point], icon=folium.Icon(color='darkblue', icon='tint')).add_to(map1)


folium.Marker(location =[location.latitude, location.longitude],
                    icon = folium.Icon(color = 'gray', icon = 'home'),).add_to(map1)


map1

Another data visualization is useful for people looking for a place to swim and not willing to pay an entrance fee. Places without entrance fee are marked as green circles, while places with entrance fee of any value are marked as red circles.

In [741]:
#preparing data with needed columns
swim_entr = swim_loc[['Name','Latitude','Longitude', 'Entrance']]

#creating map object
map_entr = folium.Map(
    location=[49.8038, 15.4749],
    tiles='cartodbpositron',
    zoom_start=8
    )

#creating a dictionary for the colors of markers
colordict = {'Entrance fee': 'red', 'No entrance fee': 'lightgreen'}

#setting CircleMarker parameters
for lat, lon, name, entrance in zip(swim_entr['Latitude'], swim_entr['Longitude'], swim_entr['Name'], swim_entr['Entrance']):
    folium.CircleMarker(
        location = [lat, lon],
        popup = ('Name: ' + str(name).capitalize() + '<br>' 'Entrance: ' + str(entrance)),
        color = 'b',
        fill_color = colordict[entrance],
        fill = True,
        fill_opacity = 0.8
        ).add_to(map_entr)

#visualization of map
map_entr

Since there are people who enjoy swimming and catching the sun without their clothes, we prepared a map representing locations to swim that are suitable for nudists. Places suitable for nudists are marked as pink circles and places not suitable for nudists are marked as dark red circles.

In [742]:
#preparing data with needed columns
swim_nudist = swim_loc[['Name','Latitude','Longitude', 'Average rating', 'Nudist beach']]

#creating map object
map_nudist = folium.Map(
    location=[49.8038, 15.4749],
    tiles='cartodbpositron',
    zoom_start=8
    )

#creating a dictionary for the colors of markers
colordict = {'Suitable for nudists': 'pink', 'Not suitable for nudists': 'darkred'}

#setting CircleMarker parameters
for lat, lon, name, nudist in zip(swim_nudist['Latitude'], swim_nudist['Longitude'], swim_nudist['Name'], swim_nudist['Nudist beach']):
    folium.CircleMarker(
        location = [lat, lon],
        popup = ('Name: ' + str(name).capitalize() + '<br>' 'Nudist beach: ' + str(nudist)),
        color = 'b',
        fill_color = colordict[nudist],
        fill = True,
        fill_opacity = 0.8
        ).add_to(map_nudist)

#visualization of map
map_nudist

In [763]:
#preparing data with needed columns
swim_rating = swim_loc[['Name','Latitude','Longitude', 'Average rating']]

#rounding column with average rating to whole numbers
av_rat = swim_rating[['Average rating']]
swim_rating.loc[:, av_rat.columns] = np.round(av_rat)


#replacing NaN values with "x"
swim_rating.loc[np.isnan(swim_rating["Average rating"]), 'Average rating'] = 'x'


#creating map object
map_rating = folium.Map(
    location=[49.8038, 15.4749],
    tiles='cartodbpositron',
    zoom_start=8
    )

#creating a dictionary for the colors of markers
colordict = {'x': 'grey',
             5: 'seagreen', 
             4: 'springgreen', 
             3: 'orange',
             2: 'red',
             1: 'firebrick'
            }

#setting CircleMarker parameters
for lat, lon, name, rating in zip(swim_rating['Latitude'], swim_rating['Longitude'], swim_rating['Name'], swim_rating['Average rating']):
    folium.CircleMarker(
        location = [lat, lon],
        popup = ('Name: ' + str(name).capitalize() + '<br>' 'Average rating: ' + str(rating)),
        color = 'b',
        fill_color = colordict[rating],
        fill = True,
        fill_opacity = 0.5
        ).add_to(map_rating)

#visualization of map
map_rating

__Next steps:__
- improve the Class
- Clean dataframe & check validity
- Analysis of data:
    - Descriptive stats
    - Visualization